In [19]:
#from arcgis.gis import GIS
#import arcgis
import numpy as np
import pandas as pd
#import requests
#from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
#import csv
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
gecko_path = 'C:/Anaconda/geckodriver.exe'
import datetime as dt
options = webdriver.firefox.options.Options()
import sqlalchemy
import pyodbc
import warnings 
warnings.filterwarnings('ignore')
from sqlalchemy import create_engine
engine=create_engine('mssql+pyodbc://ClarityProd')
engine_DW =create_engine('mssql+pyodbc://prodedw')
options.headless = False
import urllib
#from sqlalchemy.sql import text
#driver = webdriver.Firefox(options = options, executable_path = gecko_path)

################### 
def map_scraping():

    #steps to get county level data
    #create driver and open the webpage to start from
    driver = webdriver.Firefox(options = options, executable_path = gecko_path)
    driver.maximize_window()
    #driver = webdriver.Ie(options = options, executable_path = gecko_path)
    URL = 'https://www.arcgis.com/home/webmap/viewer.html?webmap=9142e7392027460dace292c6cd7fffae'
    driver.get(URL)
    driver.implicitly_wait(20)

    #click on the Content label
    content =ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'webmap-details-legend-content_label')))
    content.click()
    # then COVID19MAP5 label
    covidmap15 =ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'Covid19MapV5_1210_title')))
    covidmap15.click()
    # then Total Positive Case label 
    totalPositive =ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'Covid19MapV5_1210_0_title')))
    totalPositive.click()
    # then show table label
    tablelabel =ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'Covid19MapV5_1210_0_tableTool')))
    tablelabel.click()
    #click on the + button
    python_button = driver.find_elements_by_xpath("//button[@class='ui-icon dgrid-hider-toggle'and @aria-label='Show or hide columns']")[0]
    python_button.click()

    #select all columns
    driver.find_element_by_id('dgrid_0-hider-menu-check-all').click()
    wait =WebDriverWait(driver, 5)

    # then click again to de-select all columns
    driver.find_element_by_id('dgrid_0-hider-menu-check-all').click()
    wait =WebDriverWait(driver, 5)

    #select the 8 columns that we are interested in 
    driver.find_element_by_id('dgrid_0-hider-menu-check-0').click() #Countyfp
    driver.find_element_by_id('dgrid_0-hider-menu-check-1').click() #GEOID
    driver.find_element_by_id('dgrid_0-hider-menu-check-2').click() #County

    driver.find_element_by_id('dgrid_0-hider-menu-check-22').click() #objectid

    driver.find_element_by_id('dgrid_0-hider-menu-check-6').click() #Total positive cases

    driver.find_element_by_id('dgrid_0-hider-menu-check-10').click() #total tested

    driver.find_element_by_id('dgrid_0-hider-menu-check-8').click() #total tested not detected

    driver.find_element_by_id('dgrid_0-hider-menu-check-12').click() #total death

    #double click on the + button to de-select it 
    python_button.click()

    x =driver.find_element_by_id('dijit_layout_ContentPane_5').text

    full_data =x.split('\n')
    a=[]
    for i in range(1,int(len(full_data)/8) +1):
        b =[]
        for j in range (8):
            b.append(full_data[(i-1)*8 +j])
        a.append(b)   
    county_df=pd.DataFrame(a, columns =a[0]).iloc[1:,].sort_values(by='Name')
    
    #check if the total number of county is 93, if so, pass to next step, if not, get Phelps data separately
    cyl =1
    while cyl<5:
        if len(county_df)==93:
            driver.close()
            break
        else:

            #len(full_data)
            driver.close()
            driver = webdriver.Firefox(options = options, executable_path = gecko_path)
            driver.maximize_window()
            #driver = webdriver.Ie(options = options, executable_path = gecko_path)
            URL = 'https://www.arcgis.com/home/webmap/viewer.html?webmap=9142e7392027460dace292c6cd7fffae'
            driver.get(URL)
            driver.implicitly_wait(20)

            #click on the Content label
            content =ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'webmap-details-legend-content_label')))
            content.click()
            # then COVID19MAP5 label
            covidmap15 =ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'Covid19MapV5_1210_title')))
            covidmap15.click()
            # then Total Positive Case label 
            totalPositive =ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'Covid19MapV5_1210_0_title')))
            totalPositive.click()
            # then show table label
            tablelabel =ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'Covid19MapV5_1210_0_tableTool')))
            tablelabel.click()
            ###########'Value for County Phelps is always missing from the table, so have to scrap data for this single county'

            # click on the options button
            driver.find_element_by_class_name('esri-feature-table-button').click()

            #click on the filter selection, there are four filter option in html, this filter is the 4th one
            driver.find_elements_by_xpath("//td[contains(.,'" + 'Filter' + "') and @class='dijitReset dijitMenuItemLabel']")[3].click()

            driver.find_elements_by_class_name('dijitArrowButtonInner')[9].click()
            driver.find_element_by_id('expr_1.fieldsList_popup3').click()
            #CountyFP is 137 for phelps county
            driver.find_element_by_id('expr_1.value').send_keys('Phelps')
            #sleep for 5 second to make the FilterButton clickable
            time.sleep(5)

            #Apply the Filter 
            button = driver.find_element_by_id('applyFilterButton')
            driver.execute_script("arguments[0].click();", button)
            time.sleep(5)
            #ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'applyFilterButton'))).click()

            #Apply the Filter 
            #applyfilter =ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'applyFilterButton')))
            #applyfilter.click()

            #driver.find_element_by_id('applyFilterButton_label').sendKeys(Keys.RETURN)

            #click on the + button
            python_button = driver.find_elements_by_xpath("//button[@class='ui-icon dgrid-hider-toggle'and @aria-label='Show or hide columns']")[0]
            python_button.click()

            #select all columns
            driver.find_element_by_id('dgrid_1-hider-menu-check-all').click()
            wait =WebDriverWait(driver, 5)

            # then click again to de-select all columns
            driver.find_element_by_id('dgrid_1-hider-menu-check-all').click()
            wait =WebDriverWait(driver, 5)

            #select the 8 columns that we are interested in 
            driver.find_element_by_id('dgrid_1-hider-menu-check-0').click() #Countyfp
            driver.find_element_by_id('dgrid_1-hider-menu-check-1').click() #GEOID
            driver.find_element_by_id('dgrid_1-hider-menu-check-2').click() #County

            driver.find_element_by_id('dgrid_1-hider-menu-check-22').click() #objectid

            driver.find_element_by_id('dgrid_1-hider-menu-check-6').click() #Total positive cases

            driver.find_element_by_id('dgrid_1-hider-menu-check-10').click() #total tested

            driver.find_element_by_id('dgrid_1-hider-menu-check-8').click() #total tested not detected

            driver.find_element_by_id('dgrid_1-hider-menu-check-12').click() #total death

            #double click on the + button to de-select it 
            python_button.click()

            phelps =driver.find_element_by_id('dijit_layout_ContentPane_10').text
            #driver.close()

            full_data =phelps.split('\n')
            a=[]
            for i in range(1,int(len(full_data)/8) +1):
                b =[]
                for j in range (8):
                    b.append(full_data[(i-1)*8 +j])
                a.append(b)   
            phelps_df=pd.DataFrame(a, columns =a[0]).iloc[1:,].sort_values(by='Name')

            #concat Phelps county data with other counties' data together 
            #display(county_df.head())
            phelps_df
            county_df =pd.concat([county_df,phelps_df], ignore_index=True)
            
            cyl+=1

    #print('total county:'+ str(len(county_df)))

    county_df['Total Positive Cases'] =county_df['Total Positive:'] .str.replace(',','',case =False)

    county_df['Total Positive Cases'] =county_df['Total Positive Cases'].astype(int)
    county_df['Total Tested'] =county_df['Total Tested:'] .str.replace(',','',case =False)
    county_df['Total Tested'] =county_df['Total Tested'].astype(int)
    county_df['Total Tested Not Detected'] =county_df['*Total Tested Not Detected:'] .str.replace(',','',case =False)
    county_df['Total Tested Not Detected'] =county_df['Total Tested Not Detected'].astype(int)
    county_df['Total Deaths'] =county_df['Total Deaths:'] .str.replace(',','',case =False)
    county_df['Total Deaths'] =county_df['Total Deaths'].astype(int)
    #county_df.to_csv('C:\MyData\Project\COV_19 Forecasting\DHHS_webscrapper\data\c7ountydata.csv')
    #driver.close()
    #based on when this script is run, date will be different
    #since data on dashboard is refreshed at 6: pm daily, and there is a two-day lag with the dashboard. 
    #if run on every night after 8 but before midnight, the date will be yesterday, because data on the dashboard is from yesterday
    #if run from anytime after midnight to 5 pm or 6 pm, the date will be for two day before yesterday  
    county_df['Date'] =dt.date.today()-dt.timedelta(days=1)
    county_df=county_df[['COUNTYFP', 'GEOID', 'Name', 
            'Total Positive Cases', 'Total Tested',
           'Total Tested Not Detected', 'Total Deaths', 'Date']]
    return county_df

####repeat the scraping for at most 4 time to make sure that the number of county returned are 93
iter =1
while iter <5:
    county_df =map_scraping()
    if len(county_df) == 93:
        break
    else:
        iter+=1
        

##################### get the three total numbers from dashabord
driver = webdriver.Firefox(options = options, executable_path = gecko_path)
driver.maximize_window()
URL = 'https://nebraska.maps.arcgis.com/apps/opsdashboard/index.html#/26d5a0dac95449d7813c9323d7a41c36'
driver.get(URL)
driver.implicitly_wait(30)

totalPositive =int(driver.find_elements_by_class_name ('responsive-text-group')[1].text.replace(',',''))
totalTested=int(driver.find_elements_by_class_name ('responsive-text-group')[3].text.replace(',',''))
totalDeath =int(driver.find_elements_by_class_name ('responsive-text-group')[8].text.replace(',',''))
driver.close()

#calculate the difference between the sum of all counties and the total numbers from dashboard
PositivePending =totalPositive-county_df.groupby('Date')['Total Positive Cases'].sum()[0]
TestedPending =totalTested-county_df.groupby('Date')['Total Tested'].sum()[0]
DeathPending =totalDeath-county_df.groupby('Date')['Total Deaths'].sum()[0]
#display(TestedPending,PositivePending,DeathPending,totalTested,totalPositive,totalDeath)

pendrow ={'COUNTYFP':'','GEOID':'',
          'Name':'PENDING',
          'Total Positive Cases':PositivePending,
         'Total Tested':TestedPending,
          'Total Tested Not Detected':'',
          'Total Deaths':DeathPending,
          #'OBJECTID':'',
          'Date':dt.date.today()-dt.timedelta(days=1)}
          #'Date':dt.date.today()}

county_df=county_df.append(pendrow, ignore_index=True)
#county_df.to_csv('C:\MyData\Project\COV_19 Forecasting\DHHS_webscrapper\data\c9ountydata.csv')



########### insert into db table
county_selected =county_df[['COUNTYFP', 'GEOID', 'Name', 'Total Positive Cases', 'Total Tested',
       'Total Deaths', 'Date']]

county_selected['PullDate']=dt.date.today() 

county_selected.columns =['COUNTYFP', 'GEOID', 'County', 'Positive', 'Tested',
       'Death', 'DataCurrentDate', 'PullDate']

county_selected=county_selected[['COUNTYFP', 'GEOID', 'County', 'DataCurrentDate', 'Tested', 'Positive',
       'Death','PullDate']]

county_selected['DataCurrentDate'] =pd.to_datetime(county_selected.DataCurrentDate)

driver='{ODBC Driver 13 for SQL Server}'
server='EDWPROD'
database='BI_Reporting'
table ='NDHHS_COVID_Test_result'

#params = 'DRIVER='+driver + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';UID=' + username + ';PWD=' + password
params = 'DRIVER='+driver + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';Trusted_Connection=yes'

db_params = urllib.parse.quote_plus(params)
#make sure that version of sqlalchemy is mot up to date to be able to use fast_executemany options
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params), fast_executemany = True)
#df is the dataframe; test is table name in which this dataframe is #inserted
county_selected.to_sql(table,engine,index=False,if_exists="append",schema="dbo")